In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
df = spark.read.options(header=True, inferSchema=True).csv('data/kddcup99.csv')
df.limit(10).toPandas()

21/11/09 18:41:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/09 18:41:32 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,labels
0,0,b'tcp',b'http',b'SF',181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.00,0.0,0.0,0.0,0.0,b'normal.'
1,0,b'tcp',b'http',b'SF',239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.00,0.0,0.0,0.0,0.0,b'normal.'
2,0,b'tcp',b'http',b'SF',235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.00,0.0,0.0,0.0,0.0,b'normal.'
3,0,b'tcp',b'http',b'SF',219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.00,0.0,0.0,0.0,0.0,b'normal.'
4,0,b'tcp',b'http',b'SF',217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.00,0.0,0.0,0.0,0.0,b'normal.'
5,0,b'tcp',b'http',b'SF',217,2032,0,0,0,0,...,59,1.0,0.0,0.02,0.00,0.0,0.0,0.0,0.0,b'normal.'
6,0,b'tcp',b'http',b'SF',212,1940,0,0,0,0,...,69,1.0,0.0,1.00,0.04,0.0,0.0,0.0,0.0,b'normal.'
7,0,b'tcp',b'http',b'SF',159,4087,0,0,0,0,...,79,1.0,0.0,0.09,0.04,0.0,0.0,0.0,0.0,b'normal.'
8,0,b'tcp',b'http',b'SF',210,151,0,0,0,0,...,89,1.0,0.0,0.12,0.04,0.0,0.0,0.0,0.0,b'normal.'
9,0,b'tcp',b'http',b'SF',212,786,0,0,0,1,...,99,1.0,0.0,0.12,0.05,0.0,0.0,0.0,0.0,b'normal.'


In [2]:
# minimal config requires target and features
target_column = 'dst_host_srv_count'
feature_columns = None

In [3]:
from pyspark.sql.types import NumericType

target_is_regression = isinstance(df.schema[target_column].dataType, NumericType)
if feature_columns is None:
    feature_columns = [c for c in df.columns if c != target_column]
print(
    f'Target: {target_column} (is regression? {target_is_regression}), '
    f'num_features: {len(feature_columns)}'
)

Target: dst_host_srv_count (is regression? True), num_features: 41


In [4]:
from faas.utils_dataframe import get_non_numeric_columns

categorical_features = get_non_numeric_columns(df.select(*feature_columns))
numeric_features = [c for c in feature_columns if c not in categorical_features]
print(
    f'num_features: {len(feature_columns)} '
    f'num_categorical_features: {len(categorical_features)} '
    f'num_numeric_features: {len(numeric_features)}'
)

num_features: 41 num_categorical_features: 4 num_numeric_features: 37


In [5]:
from faas.base import Pipeline
from faas.scaler import StandardScaler
from faas.encoder import OrdinalEncoder

# not minimal, just testing
target_group = 'protocol_type'
getx = Pipeline(steps=[
    OrdinalEncoder(c)
    for c in categorical_features
]).fit(df)
gety = Pipeline(steps=[
    StandardScaler(column=target_column, group_column=target_group)
]).fit(df)


In [6]:
X = (
    getx
    .transform(df)
    .select(getx.feature_columns + numeric_features)
    .toPandas()
)
X.head()

,OrdinalEncoder_protocol_type,OrdinalEncoder_service,OrdinalEncoder_flag,OrdinalEncoder_labels,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,...,srv_diff_host_rate,dst_host_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,1,9,0,1,0,181,5450,0,0,0,...,0.0,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0
1,1,9,0,1,0,239,486,0,0,0,...,0.0,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0
2,1,9,0,1,0,235,1337,0,0,0,...,0.0,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
3,1,9,0,1,0,219,1337,0,0,0,...,0.0,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
4,1,9,0,1,0,217,2032,0,0,0,...,0.0,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0


In [7]:
y = (
    gety
    .transform(df)
    .select(gety.feature_columns)
    .toPandas()
)
y.head()

,StandardScaler_dst_host_srv_count_by_protocol_type
0,9.0
1,19.0
2,29.0
3,39.0
4,49.0


In [8]:
from lightgbm import LGBMModel

m = (
    LGBMModel(
        objective='regression' if target_is_regression else 'binary'
    )
    .fit(X=X, y=y)
)

In [9]:
from faas.utils_dataframe import JoinableByRowID

jb = JoinableByRowID(df)
Xpred = (
    getx
    .transform(jb.df)
    .select(getx.feature_columns + numeric_features)
    .toPandas()
)
ypred = m.predict(Xpred)

In [10]:
gety.transform(jb.df).limit(10).toPandas()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,labels,__ROW_ID__,StandardScaler_dst_host_srv_count_by_protocol_type
0,0,b'tcp',b'http',b'SF',181,5450,0,0,0,0,...,0.0,0.11,0.00,0.0,0.0,0.0,0.0,b'normal.',0,9.0
1,0,b'tcp',b'http',b'SF',239,486,0,0,0,0,...,0.0,0.05,0.00,0.0,0.0,0.0,0.0,b'normal.',1,19.0
2,0,b'tcp',b'http',b'SF',235,1337,0,0,0,0,...,0.0,0.03,0.00,0.0,0.0,0.0,0.0,b'normal.',2,29.0
3,0,b'tcp',b'http',b'SF',219,1337,0,0,0,0,...,0.0,0.03,0.00,0.0,0.0,0.0,0.0,b'normal.',3,39.0
4,0,b'tcp',b'http',b'SF',217,2032,0,0,0,0,...,0.0,0.02,0.00,0.0,0.0,0.0,0.0,b'normal.',4,49.0
5,0,b'tcp',b'http',b'SF',217,2032,0,0,0,0,...,0.0,0.02,0.00,0.0,0.0,0.0,0.0,b'normal.',5,59.0
6,0,b'tcp',b'http',b'SF',212,1940,0,0,0,0,...,0.0,1.00,0.04,0.0,0.0,0.0,0.0,b'normal.',6,69.0
7,0,b'tcp',b'http',b'SF',159,4087,0,0,0,0,...,0.0,0.09,0.04,0.0,0.0,0.0,0.0,b'normal.',7,79.0
8,0,b'tcp',b'http',b'SF',210,151,0,0,0,0,...,0.0,0.12,0.04,0.0,0.0,0.0,0.0,b'normal.',8,89.0
9,0,b'tcp',b'http',b'SF',212,786,0,0,0,1,...,0.0,0.12,0.05,0.0,0.0,0.0,0.0,b'normal.',9,99.0


In [11]:
dfpred = jb.join_by_row_id(ypred, column=gety.feature_columns[0])
dfpred.limit(10).toPandas()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,labels,__ROW_ID__,StandardScaler_dst_host_srv_count_by_protocol_type
0,0,b'tcp',b'http',b'SF',181,5450,0,0,0,0,...,0.0,0.11,0.00,0.0,0.0,0.0,0.0,b'normal.',0,30.396071
1,0,b'tcp',b'http',b'SF',239,486,0,0,0,0,...,0.0,0.05,0.00,0.0,0.0,0.0,0.0,b'normal.',1,19.077804
2,0,b'tcp',b'http',b'SF',235,1337,0,0,0,0,...,0.0,0.03,0.00,0.0,0.0,0.0,0.0,b'normal.',2,32.983221
3,0,b'tcp',b'http',b'SF',219,1337,0,0,0,0,...,0.0,0.03,0.00,0.0,0.0,0.0,0.0,b'normal.',3,41.487570
4,0,b'tcp',b'http',b'SF',217,2032,0,0,0,0,...,0.0,0.02,0.00,0.0,0.0,0.0,0.0,b'normal.',4,53.087178
5,0,b'tcp',b'http',b'SF',217,2032,0,0,0,0,...,0.0,0.02,0.00,0.0,0.0,0.0,0.0,b'normal.',5,54.647201
6,0,b'tcp',b'http',b'SF',212,1940,0,0,0,0,...,0.0,1.00,0.04,0.0,0.0,0.0,0.0,b'normal.',6,141.903312
7,0,b'tcp',b'http',b'SF',159,4087,0,0,0,0,...,0.0,0.09,0.04,0.0,0.0,0.0,0.0,b'normal.',7,150.469136
8,0,b'tcp',b'http',b'SF',210,151,0,0,0,0,...,0.0,0.12,0.04,0.0,0.0,0.0,0.0,b'normal.',8,156.119297
9,0,b'tcp',b'http',b'SF',212,786,0,0,0,1,...,0.0,0.12,0.05,0.0,0.0,0.0,0.0,b'normal.',9,127.588959
